# Feed-Forward Neural Network

We will be running a feed-forward neural network on the MNIST dataset available in the Pytorch library to demonstrate the easy of creating such a machine learning architecture using Pytorch.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# hyper parameters
input_size = 784 # image has size 28 x 28, when flattened becomes 784
hidden_size = 100
num_classes = 10 # 10 different classes
num_epochs = 2 # can set to higher value
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST
training_dataset = torchvision.datasets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)
test_dataset = torchvision.datasets.MNIST(root = './data', train = False, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = training_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
model = NeuralNet(input_size, hidden_size, num_classes)

In [6]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [7]:
# Training Loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 100,1,28,28
        # 100,784
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/2, step 100/600, loss = 0.3927
epoch 1/2, step 200/600, loss = 0.3903
epoch 1/2, step 300/600, loss = 0.4412
epoch 1/2, step 400/600, loss = 0.4549
epoch 1/2, step 500/600, loss = 0.3342
epoch 1/2, step 600/600, loss = 0.2251
epoch 2/2, step 100/600, loss = 0.2110
epoch 2/2, step 200/600, loss = 0.1971
epoch 2/2, step 300/600, loss = 0.1866
epoch 2/2, step 400/600, loss = 0.3475
epoch 2/2, step 500/600, loss = 0.1283
epoch 2/2, step 600/600, loss = 0.1107


In [8]:
# Test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # value, index (class label)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
    acc = 100.0  * n_correct/n_samples
    print(f'accuracy = {acc}')

accuracy = 95.17


Even in just 2 epochs, we are able to reach a 95% accuracy.